In [ ]:
# RESET ENVIRONMENT & FIX VERSIONS
!pip uninstall -y mediapipe protobuf numpy
!pip install mediapipe==0.10.21 numpy==1.26.4 protobuf==4.25.6

import os
os.kill(os.getpid(), 9)  # This will RESTART the runtime so versions are correctly loaded


Found existing installation: protobuf 5.29.4
Uninstalling protobuf-5.29.4:
  Successfully uninstalled protobuf-5.29.4
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 28.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.6 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.6 which is

In [ ]:
import cv2
import numpy as np
import mediapipe as mp

print("✅ All imports successful!")


✅ All imports successful!


In [ ]:
from google.colab import files
files.upload()  # Upload the kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tejaslakhera","key":"4b2b1391443bf14880e32233b86fddfa"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#for training
!kaggle datasets download -d rushilverma07/indian-sign-language-alphabet-dataset
!unzip -q indian-sign-language-alphabet-dataset.zip -d isl_dataset_train

#for testing
!kaggle datasets download -d prathumarikeri/indian-sign-language-isl
!unzip -q indian-sign-language-isl.zip -d isl_dataset_test


Dataset URL: https://www.kaggle.com/datasets/rushilverma07/indian-sign-language-alphabet-dataset
License(s): Attribution-NoDerivatives 4.0 International (CC BY-ND 4.0)
Dataset URL: https://www.kaggle.com/datasets/prathumarikeri/indian-sign-language-isl
License(s): CC-BY-SA-4.0


In [ ]:
!ls isl_dataset/dataset

'{'   b   d   f   h   j   l   n   p   r   t   v   x   z
 a    c   e   g   i   k   m   o   q   s   u   w   y


In [ ]:
import os
import shutil
import random

# Define unwanted classes
UNWANTED_CLASSES = {"{", "1", "2", "3", "4", "5", "6", "7", "8", "9"}

def create_filtered_subset(source_dir, target_dir, max_images_per_class, unwanted_classes):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if not os.path.isdir(class_path) or class_name in unwanted_classes:
            continue

        files = os.listdir(class_path)
        random.shuffle(files)
        selected_files = files[:max_images_per_class]

        dest_class_path = os.path.join(target_dir, class_name)
        os.makedirs(dest_class_path, exist_ok=True)

        for file in selected_files:
            src = os.path.join(class_path, file)
            dst = os.path.join(dest_class_path, file)
            shutil.copy(src, dst)

# Dataset folder paths
train_dataset = "isl_dataset_train/dataset"
test_dataset = "isl_dataset_test/dataset"

# Output folders
train_output = "ISL_cleaned_train"
test_output = "ISL_cleaned_test"

# Create cleaned subsets
create_filtered_subset(train_dataset, train_output, 200, UNWANTED_CLASSES)
create_filtered_subset(test_dataset, test_output, 50, UNWANTED_CLASSES)


In [ ]:
!ls ISL_cleaned_train
!ls ISL_cleaned_test

a  b  c  d  e  f  g  h	i  j  k  l  m  n  o  p	q  r  s  t  u  v  w  x	y  z
A  B  C  D  E  F  G  H	I  J  K  L  M  N  O  P	Q  R  S  T  U  V  W  X	Y  Z


In [ ]:
def rename_folders_to_lowercase(root_dir):
    for folder in os.listdir(root_dir):
        full_path = os.path.join(root_dir, folder)
        if os.path.isdir(full_path):
            lower_name = folder.lower()
            if folder != lower_name:
                os.rename(full_path, os.path.join(root_dir, lower_name))


rename_folders_to_lowercase("ISL_cleaned_train")
rename_folders_to_lowercase("ISL_cleaned_test")


In [ ]:
!ls ISL_cleaned_train
!ls ISL_cleaned_test

a  b  c  d  e  f  g  h	i  j  k  l  m  n  o  p	q  r  s  t  u  v  w  x	y  z
a  b  c  d  e  f  g  h	i  j  k  l  m  n  o  p	q  r  s  t  u  v  w  x	y  z


In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np
from tqdm import tqdm

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

def crop_hand(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        h, w, _ = img.shape
        hand_landmarks = results.multi_hand_landmarks[0]
        x_coords = [lm.x for lm in hand_landmarks.landmark]
        y_coords = [lm.y for lm in hand_landmarks.landmark]

        x_min = int(min(x_coords) * w) - 20
        x_max = int(max(x_coords) * w) + 20
        y_min = int(min(y_coords) * h) - 20
        y_max = int(max(y_coords) * h) + 20

        x_min = max(0, x_min)
        y_min = max(0, y_min)
        x_max = min(w, x_max)
        y_max = min(h, y_max)

        cropped = img[y_min:y_max, x_min:x_max]
        if cropped.size == 0:
            return img
        return cropped
    return img

def process_folder(input_dir, output_dir, target_size=(128, 128)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_folder in tqdm(os.listdir(input_dir)):
        class_input_path = os.path.join(input_dir, class_folder)
        class_output_path = os.path.join(output_dir, class_folder)
        os.makedirs(class_output_path, exist_ok=True)

        for img_file in os.listdir(class_input_path):
            img_path = os.path.join(class_input_path, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            cropped = crop_hand(img)
            resized = cv2.resize(cropped, target_size)
            out_path = os.path.join(class_output_path, img_file)
            cv2.imwrite(out_path, resized)

# Example usage
process_folder('/content/ISL_cleaned_train', '/content/isl_processed_train')
process_folder('/content/ISL_cleaned_test', '/content/isl_processed_test')


100%|██████████| 26/26 [00:38<00:00,  1.49s/it]


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import json

train_dir = '/content/isl_processed_train'
val_dir = '/content/isl_processed_test'

train_gen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    shear_range=0.15,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

Found 5200 images belonging to 26 classes.
Found 1300 images belonging to 26 classes.


In [ ]:
with open("isl_class_indices_mediapipe.json", "w") as f:
    json.dump(train_data.class_indices, f)

In [ ]:
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)

model.fit(train_data, validation_data=val_data, epochs=10, callbacks=[early_stop, reduce_lr])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 45s 212ms/step - accuracy: 0.0763 - loss: 3.3997 - val_accuracy: 0.1738 - val_loss: 2.9210 - learning_rate: 1.0000e-04
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 151ms/step - accuracy: 0.3258 - loss: 2.4343 - val_accuracy: 0.2638 - val_loss: 2.3991 - learning_rate: 1.0000e-04
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 24s 148ms/step - accuracy: 0.4703 - loss: 1.8660 - val_accuracy: 0.3946 - val_loss: 2.0451 - learning_rate: 1.0000e-04
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 24s 149ms/step - accuracy: 0.6009 - loss: 1.4966 - val_accuracy: 0.4285 - val_loss: 1.9043 - learning_rate: 1.0000e-04
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 150ms/step - accuracy: 0.6320 - loss: 1.3150 - val_accuracy: 0.4477 - val_loss: 1.7347 - learning_rate: 1.0000e-04
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 24s 149ms/step - accuracy: 0.6767 - loss: 1.1632 - val_accuracy: 0.4646 - val_loss: 1.6309 - learning_rate: 1.0000e-04
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 24s 14

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:80]:
    layer.trainable = False

In [ ]:
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_data, validation_data=val_data, epochs=15, callbacks=[early_stop, reduce_lr])

Epoch 1/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 64s 239ms/step - accuracy: 0.3241 - loss: 2.4226 - val_accuracy: 0.5308 - val_loss: 1.5125 - learning_rate: 1.0000e-05
Epoch 2/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 152ms/step - accuracy: 0.5453 - loss: 1.5223 - val_accuracy: 0.5092 - val_loss: 1.5976 - learning_rate: 1.0000e-05
Epoch 3/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 154ms/step - accuracy: 0.6759 - loss: 1.1222 - val_accuracy: 0.5062 - val_loss: 1.6054 - learning_rate: 1.0000e-05
Epoch 4/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 155ms/step - accuracy: 0.7489 - loss: 0.8947 - val_accuracy: 0.5046 - val_loss: 1.5537 - learning_rate: 1.0000e-05
Epoch 5/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 153ms/step - accuracy: 0.7754 - loss: 0.7607 - val_accuracy: 0.5100 - val_loss: 1.5334 - learning_rate: 2.0000e-06
Epoch 6/15
163/163 ━━━━━━━━━━━━━━━━━━━━ 25s 151ms/step - accuracy: 0.7716 - loss: 0.7797 - val_accuracy: 0.5115 - val_loss: 1.5543 - learning_rate: 2.0000e-06


In [ ]:
model.save('/content/drive/MyDrive/models/isl_mediapipe_model.h5')

In [ ]:
model.save('/content/drive/MyDrive/models/isl_mediapipe_model.keras')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("/content/isl_mediapipe_model.keras", compile=False)


In [ ]:
import json

with open("/content/isl_class_indices_mediapipe.json", "r") as f:
    label_map = json.load(f)
    print("Original label map:", label_map)

reverse_map = {v: k for k, v in label_map.items()}
print("Reversed label map:", reverse_map)

Original label map: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}
Reversed label map: {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}


In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f"Test Accuracy: {accuracy:.2f}")


28/28 ━━━━━━━━━━━━━━━━━━━━ 11s 385ms/step - accuracy: 0.4645 - loss: 1.8241
Test Accuracy: 0.47


In [ ]:
print(train_data.class_indices)
print(val_data.class_indices)


{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, '{': 26}
{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, '{': 26}


In [ ]:
import cv2
import numpy as np
import json
from tensorflow.keras.models import load_model

# Load model
model = load_model("/content/isl_mediapipe_model.h5")

# Load label map
with open("/content/isl_class_indices_mediapipe.json", "r") as f:
    label_map = json.load(f)
    label_map = {v: k for k, v in label_map.items()}  # Reverse map: index -> label

# Load and preprocess image
img = cv2.imread("/content/a_isl.png")  # Double-check file name/path
if img is None:
    raise FileNotFoundError("Image not found. Make sure 'b_isl.jpg' is in the correct path.")

img = cv2.resize(img, (128, 128)) / 255.0  # Use 224x224 if that's what model was trained on
img = np.expand_dims(img, axis=0)

# Predict
prediction = model.predict(img)
pred_index = np.argmax(prediction)
pred_label = label_map[pred_index]

print("Prediction:", pred_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Prediction: a


In [ ]:
print("Prediction:", pred_label, "| Confidence:", np.max(prediction))


Prediction: M | Confidence: 0.83200073


In [ ]:
prediction = model.predict(img)
for i, prob in enumerate(prediction[0]):
    print(f"{label_map[i]}: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
A: 0.0031
B: 0.0001
C: 0.0001
D: 0.0001
E: 0.0000
F: 0.0007
G: 0.0025
H: 0.0003
I: 0.0002
J: 0.0002
K: 0.0000
L: 0.0006
M: 0.8320
N: 0.0138
O: 0.0001
P: 0.0000
Q: 0.0000
R: 0.0001
S: 0.0002
T: 0.0381
U: 0.0000
V: 0.0001
W: 0.0001
X: 0.0005
Y: 0.1064
Z: 0.0001
del: 0.0004
nothing: 0.0000
space: 0.0002
